In [2]:
# import datasets : 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
data_yelp = pd.read_csv('yelp-labelled.txt',sep='\t',header=None)

In [5]:
data_yelp.head()

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [6]:
column_name = ['Review','Sentiment']
data_yelp.columns = column_name # set Column Name

In [7]:
# data_yelp.head()
data_yelp.shape

(1000, 2)

In [8]:
data_amazon = pd.read_csv('amazon-cells-labelled.txt',sep='\t',header=None)

In [12]:
data_amazon.columns = column_name

In [13]:
data_imdb = pd.read_csv('imdb-labelled.txt',sep='\t',header=None)

In [15]:
data_imdb.columns = column_name
data_imdb.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [18]:
#append all the data in single dataframe
data = data_yelp.append([data_amazon,data_imdb],ignore_index=True)

In [19]:
data.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [20]:
data.shape

(2748, 2)

In [21]:
data['Sentiment'].value_counts()

1    1386
0    1362
Name: Sentiment, dtype: int64

In [22]:
# check for null values:
data.isnull().sum()

Review       0
Sentiment    0
dtype: int64

In [25]:
x = data['Review']
y = data['Sentiment']

In [27]:
# Data Cleaning : 
# here we will remove stopwords,punctuations
# as well as we will apply lemmatization :)
import string

In [29]:
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [32]:
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS) # list of stopwords

In [33]:
#creating a function for dataCleaning :)
import spacy
nlp = spacy.load('en_core_web_sm')
def text_data_cleaning(sentence):
  doc = nlp(sentence)
  tokens = []
  for token in doc:
    if token.lemma_ != "-PRON-":
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)
  
  cleaned_tokens = []
  for token in tokens:
    if token not in stopwords and token not in punct:
      cleaned_tokens.append(token)
  return cleaned_tokens

In [34]:
text_data_cleaning("Hello all,It's a beautiful day outside there !")

['hello', 'beautiful', 'day', 'outside']

In [39]:
# Vectorization Feature Engineering (TF-IDF)
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [40]:
tfidf = TfidfVectorizer(tokenizer=text_data_cleaning)
# tokenizer=text_data_cleaning, tokenization will be done according to this function  

In [41]:
classifier = LinearSVC()

In [43]:
# train this model : 
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [44]:
x_train.shape , x_test.shape

((2198,), (550,))

In [45]:
x_train.head()

2572    An Italian reviewer called this "a small, grea...
526                          And it was way to expensive.
1509    As an earlier review noted, plug in this charg...
144     Nice blanket of moz over top but i feel like t...
2483    The film gives meaning to the phrase, "Never i...
Name: Review, dtype: object

In [46]:
clf = Pipeline([('tfidf',tfidf),('clf',classifier)])
#it will first do vectorization and then it will do classification:)

In [48]:
clf.fit(x_train,y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x7fe38562c170>)),
                ('clf', LinearSVC())])

In [49]:
# in this we dont need to prepare the dataset for testing(x_test)

In [51]:
# predict the test set results:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
y_pred = clf.predict(x_test)

In [52]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.80      0.72      0.76       279
           1       0.74      0.81      0.77       271

    accuracy                           0.77       550
   macro avg       0.77      0.77      0.77       550
weighted avg       0.77      0.77      0.77       550



In [53]:
accuracy_score(y_test,y_pred)

0.7672727272727272

In [54]:
clf.predict(["Wow, I am learning natural language processing in fun fashion!"]) # 1 means positive

array([1])

In [55]:
clf.predict(["it's hard to lean new things!"]) # 0 means negative

array([0])